Listen to Samples that have Paraphasias or have predicted Paraphasias

In [4]:
'''
Tunnel to remote server:
1) on boba `jupyter notebook --no-browser --port=8080`
2) on local `ssh -L 8080:localhost:8080 mkperez@boba.eecs.umich.edu`
3) open up web browser on local and play
'''

import IPython.display as ipd
import pandas as pd
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import librosa
from sklearn.metrics import f1_score

def display_info(series):
    print(f"{series['utt_id']}:\nTrue: {series['true']}\nPred: {series['pred']}")
    ipd.display(ipd.Audio(filename=series['wav']))

def extract_utts(wer_file, wav_dir):
    list_ids=[]
    list_pred=[]
    list_gt=[]
    list_wavpath=[]
    
    tracker=0
    paraphasia_detector=False
    with open(wer_file, 'r') as r:
        lines = r.readlines()
        for line in lines:
            line = line.strip()
            # print(line)
            if line.startswith("P") and len(line.split())==14:
                utt_id = line.split()[0][:-1]
                tracker = 1
                paraphasia_detector=False
            elif tracker == 1:
                # GT
                gt_line = " ".join([x.strip() for x in line.split(";")])
                tracker = 2
                if '/P' in gt_line:
                    paraphasia_detector=True
            elif tracker == 2:
                # sub line
                tracker=3
            elif tracker == 3:
                # pred
                pred_line = " ".join([x.strip() for x in line.split(";")])

                tracker = 0
                if '/P' in pred_line:
                    paraphasia_detector=True
                
                # add to lists
                if paraphasia_detector:
                    list_ids.append(utt_id)
                    list_gt.append(gt_line)
                    list_pred.append(pred_line)
                    list_wavpath.append(f'{wav_dir}/{utt_id}.wav')

    return pd.DataFrame({
        'utt_id': list_ids,
        'true': list_gt,
        'pred': list_pred,
        'wav': list_wavpath
    })
            
FOLD_NUM=2
ROOT_DIR = f"/z/mkperez/speechbrain/AphasiaBank/results/Frid/100-unigram-p/Fold-{FOLD_NUM}"
WAV_DIR = "/z/mkperez/speechbrain/AphasiaBank/data/Fridriksson_para/wavs"
Para_Text = f"{ROOT_DIR}/awer_para.txt"

df = extract_utts(wer_file=Para_Text, wav_dir=WAV_DIR)

# y_true=[1,0,1,0]
# y_pred=[1,1,0,1]
# print(f"f1: {f1_score(y_true, y_pred)}")
for i in range(10):
    display_info(series=df.iloc[i])


f1: 0.4
P11_6W_SA_C6-0:
True: high/P fought/C aphasia/C <eps>
Pred: i/C fat/C uh/C aphasia/P


P11_P2_SE_C5-1:
True: like/C eggs/C eggs/C eggs/C <eps>
Pred: its/P eggs/C its/C eggs/C is/C


P11_T6_SE_C2-5:
True: and/C stir/C and/C stir/C and/C stir/C stir/P and/C stir/C and/C stir/C stir/C stir/C and/C stir/C done/C done/C and/C the/C done/C done/C
Pred: and/C stir/C <eps> stir/C <eps> stir/C <eps> <eps> stir/C <eps> stir/C stir/C stir/C <eps> stir/C stufffficulty/C <eps> and/C stir/C <eps> <eps>


P11_B2_SA_C1-0:
True: i/C have/C uh/C aphasia/C aphasia/C
Pred: i/C have/C aphasia/P <eps> <eps>


IndexError: single positional indexer is out-of-bounds